In [1974]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize 
from pandas import read_csv
import logging
from functools import reduce
import csv

## Configuration 

In [1756]:
#etl = 'kobo2elastic'
etl = 'curis2elastic'
#etl = 'oldcuris2newcuris'

input_schema_file = ''
input_data_file = ''
mapping_file = ''
    
if etl == 'curis2elastic':
    #old curis to elasticsearch
    input_schema_file = 'schema/input/curisSchema.1-item.json'
    input_data_file = 'data/curisData.1-items.json'
    mapping_file = 'schema/map/couchbase2elastic.map.csv'
elif etl == 'kobo2elastic':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/aqmSchema.complete.json'
    input_data_file = 'data/aqmData.2-items.json'
    mapping_file = 'schema/map/kobo2elastic.map.csv'
elif etl == 'oldcuris2newcuris':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/curisData.1-Schema.avro.json'
    input_data_file = 'data/curisData.1-items.json'
    mapping_file = 'schema/map/kobo2elastic.map.csv'

## Create separate CSV files based on array types

In [2242]:
main_resident_df  = pd.DataFrame(columns = main_header_list)
#main_resident_df = main_resident_df.iloc[0:0]
#main_resident_df
#main_resident_df.columns = main_header

#TODO: check if header/columns already exist
main_resident_df.to_csv('file/resident.csv', encoding='utf-8', mode='a', index=False)
main_resident_df

,birthdate,family_members,first_name,gender,id,last_name,last_name_suffix,middle_name,nhid,organization,registered_at,type,identification.id2.type,user-cam.id,user-cam.owner,profile_picture.path,identification.id3.type,identification.id1.type,profile_picture.name


## split ang get first string, 
### if string is equal to csv filename, include as csv header in same csv filename
### if string not equal, include as csv header in main csv

In [ ]:
headers = []
headers =  list(csv_header_name_df['source_key'])

empty_df = pd.DataFrame()
empty_df.columns = ["Sequence", "Start", "End", "Coverage"]
#empty_df.to_csv('file/resident.csv', encoding='utf-8', mode='a', header=None, index=False)

In [ ]:


#df.to_csv('file/resident.csv', encoding='utf-8', mode='w', header=header, index=False)


In [ ]:
d = []
with open(input_data_file) as f:
    #d = json.load(f)
    d = json.load(f)
    print('----')
    print(d)
    print('---type--')
print(type(d))

## Get input data

In [ ]:

flat_json_0 = {}
flat_json_1 = {}
flat_json_2 = {}

flat_json_0 = flattenDict(d[0])
print('--flatten_dict_type--')
print(type(flat_json_0))
print('--flatten_dict_data--')
flat_json_0

print('--normalize flatten_dict--')
json_flat_norm_0 = json_normalize(flat_json_0)


flat_json_1 = flattenDict(d[1])
json_flat_norm_1 = json_normalize(flat_json_1)

flat_json_2 = flattenDict(d[2])
json_flat_norm_2 = json_normalize(flat_json_2)



In [ ]:
json_flat_norm_0

In [ ]:
json_flat_norm_1

In [ ]:
json_flat_norm_2

## TESTING: flattendict with nested array of objects

In [ ]:
dx = []
dx = {}
dx_list = []
dx_dict = {}
dx_normalize_df = pd.DataFrame()

input_data_file_test = 'data/curisData.1-items.json'
with open(input_data_file_test) as f:
    dx_list = json.load(f)
    #dx = json.load(f)

type(dx_list)

In [ ]:
dx_flatten = flatten_json(dx_list[0])
dx_flatten

In [ ]:
dx_normalize_df = json_normalize(dx_flatten)

dx_normalize_df.T.to_csv('processed_data.csv',sep=',')
type(dx_normalize_df)

In [ ]:
list(dx_normalize_df.columns)

In [ ]:
dx_normalize_df

## TODO: ITERATE OVER LIST and MERGE normalize headers

In [ ]:
input_data_df = pd.DataFrame()
input_data_df = json_flat_norm_0
input_data_df = input_data_df.append(json_flat_norm_1, sort=True)
input_data_df = input_data_df.append(json_flat_norm_2, sort=True)
input_data_df.sort_values(['demographics.awh_id']).reset_index(drop=True)
#input_data_df.sort_values(p'd)

In [ ]:
type(input_data_df)

## Clean input data column header

In [ ]:
input_data_df.columns = input_data_df.columns.str.lower().str.replace('/','.')
input_data_df

## Get Mapping 

In [ ]:
mapping_df = pd.read_csv(mapping_file, skiprows=0)
mapping_df

## Get values input data that are included mapping

In [ ]:
selected_data_df = pd.DataFrame()
selected_data_df = input_data_df[list(mapping_df['source_key'])]
selected_data_df

## Renamce source header fields into destination header fields

In [ ]:
selected_data_df.columns = list(mapping_df['destination_key'])
selected_data_df

## Encode to as Filesystem (HDFS) or S3 (avro) format)

## Decode for cleaning

## Decode to for computation

## Decode to for analytics transformation

## Transform flat file into output schema format

In [ ]:
flat_json = ''
flat_json = selected_data_df.to_json(orient='records')
json_json = json.loads(flat_json)
json_json

## Convert dot notated fields into nested json

In [ ]:
input_file = 'data/data.json'
with open(input_file) as f:
    d = (f)
 


In [ ]:

def dot_to_json(a):
    output = {}
    for key, value in a.items():
        path = key.split('.')[1:]  # ignore the json. prefix
        #path = key
        target = reduce(lambda d, k: d.setdefault(k, {}), path[:-1], output)
        target[path[-1]] = value
    return output
 
data = {'json.message.status.time':50, 'json.message.code.response':80, 'json.time':100}
type(data)


data

## data

In [ ]:
data

## dot data

In [ ]:
dict_json = dot_to_json(data)
dict_json

## dictionary to json

In [ ]:
def _dict2json(results):
    counter = 0
    data = []

    for row in results: 
        data.append(json.dumps(row))
        counter += 1
    
    return data

json_dict = _dict2json(dict_json) 
json_dict

In [ ]:
iris = pd.DataFrame()
iris = pd.read_csv('data/sql.csv')
iris
#iris.to_json(orient='records')
list(iris['0'])

In [ ]:
iris